In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import os
import collections
collections.Callable = collections.abc.Callable

## Start by choosing what demographic variables we want to use

In [4]:
demographic_url = 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DEMO.XPT'
demographic_df = pd.read_sas(demographic_url)

In [5]:
pd.set_option('display.max_columns', 30)
demographic_df

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,DMDBORN4,DMDYRUSZ,DMDEDUC2,DMDMARTZ,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,WTINTPRP,WTMECPRP,SDMVPSU,SDMVSTRA,INDFMPIR
0,109263.0,66.0,2.0,1.0,2.0,NaN,5.0,6.0,2.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,7891.762435,8.951816e+03,3.0,156.0,4.66
1,109264.0,66.0,2.0,2.0,13.0,NaN,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,11689.747264,1.227116e+04,1.0,155.0,0.83
2,109265.0,66.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,16273.825939,1.665876e+04,1.0,157.0,3.06
3,109266.0,66.0,2.0,2.0,29.0,NaN,5.0,6.0,2.0,2.0,2.0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,7825.646112,8.154968e+03,2.0,168.0,5.00
4,109267.0,66.0,1.0,2.0,21.0,NaN,2.0,2.0,NaN,2.0,3.0,4.0,3.0,3.0,1.0,2.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,26379.991724,5.397605e-79,1.0,156.0,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,124818.0,66.0,2.0,1.0,40.0,NaN,4.0,4.0,1.0,1.0,NaN,5.0,1.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,21586.596728,2.166689e+04,1.0,166.0,3.82
15556,124819.0,66.0,2.0,1.0,2.0,NaN,4.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,1664.919253,1.838170e+03,2.0,171.0,0.07
15557,124820.0,66.0,2.0,2.0,7.0,NaN,3.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,14819.783161,1.649781e+04,1.0,157.0,1.22
15558,124821.0,66.0,2.0,1.0,63.0,NaN,4.0,4.0,1.0,1.0,NaN,2.0,2.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,4666.817952,4.853430e+03,1.0,158.0,3.71


In [6]:
demographic_df['DMDBORN4'].value_counts()

1.0     12525
2.0      3028
77.0        6
99.0        1
Name: DMDBORN4, dtype: int64

In [7]:
demographic_df['RIDRETH1'].isna().sum()

0

In [8]:
demographic_df.columns

Index(['SEQN', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN',
       'RIDRETH1', 'RIDRETH3', 'RIDEXMON', 'DMDBORN4', 'DMDYRUSZ', 'DMDEDUC2',
       'DMDMARTZ', 'RIDEXPRG', 'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG',
       'FIAPROXY', 'FIAINTRP', 'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA',
       'WTINTPRP', 'WTMECPRP', 'SDMVPSU', 'SDMVSTRA', 'INDFMPIR'],
      dtype='object')

In [172]:
cols_to_keep = ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDEDUC2', 'DMDMARTZ', 'RIDEXPRG', 'INDFMPIR']
demo_clean = demographic_df[cols_to_keep]
demo_clean

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH3,DMDEDUC2,DMDMARTZ,RIDEXPRG,INDFMPIR
0,109263.0,1.0,2.0,6.0,NaN,NaN,NaN,4.66
1,109264.0,2.0,13.0,1.0,NaN,NaN,NaN,0.83
2,109265.0,1.0,2.0,3.0,NaN,NaN,NaN,3.06
3,109266.0,2.0,29.0,6.0,5.0,3.0,2.0,5.00
4,109267.0,2.0,21.0,2.0,4.0,3.0,3.0,5.00
...,...,...,...,...,...,...,...,...
15555,124818.0,1.0,40.0,4.0,5.0,1.0,NaN,3.82
15556,124819.0,1.0,2.0,4.0,NaN,NaN,NaN,0.07
15557,124820.0,2.0,7.0,3.0,NaN,NaN,NaN,1.22
15558,124821.0,1.0,63.0,4.0,2.0,2.0,NaN,3.71


In [173]:
demo_clean = demo_clean.rename({'SEQN': 'SEQN',
                              'RIAGENDR':'sex', # 1 male, 2 female
                              'RIDAGEYR':'age_year',
                              'RIDRETH3':'race',
                              'DMDEDUC2':'education',
                              'DMDMARTZ':'marital_status',
                              'RIDEXPRG':'pregnant',
                              'INDFMPIR': 'income_pov_rat'},
                            axis=1)
demo_clean

,SEQN,sex,age_year,race,education,marital_status,pregnant,income_pov_rat
0,109263.0,1.0,2.0,6.0,NaN,NaN,NaN,4.66
1,109264.0,2.0,13.0,1.0,NaN,NaN,NaN,0.83
2,109265.0,1.0,2.0,3.0,NaN,NaN,NaN,3.06
3,109266.0,2.0,29.0,6.0,5.0,3.0,2.0,5.00
4,109267.0,2.0,21.0,2.0,4.0,3.0,3.0,5.00
...,...,...,...,...,...,...,...,...
15555,124818.0,1.0,40.0,4.0,5.0,1.0,NaN,3.82
15556,124819.0,1.0,2.0,4.0,NaN,NaN,NaN,0.07
15557,124820.0,2.0,7.0,3.0,NaN,NaN,NaN,1.22
15558,124821.0,1.0,63.0,4.0,2.0,2.0,NaN,3.71


In [223]:
demo = demo_clean.query("pregnant != 1").drop(['pregnant'], axis = 1)
demo

,SEQN,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,1.0,2.0,6.0,NaN,NaN,4.66
1,109264.0,2.0,13.0,1.0,NaN,NaN,0.83
2,109265.0,1.0,2.0,3.0,NaN,NaN,3.06
3,109266.0,2.0,29.0,6.0,5.0,3.0,5.00
4,109267.0,2.0,21.0,2.0,4.0,3.0,5.00
...,...,...,...,...,...,...,...
15555,124818.0,1.0,40.0,4.0,5.0,1.0,3.82
15556,124819.0,1.0,2.0,4.0,NaN,NaN,0.07
15557,124820.0,2.0,7.0,3.0,NaN,NaN,1.22
15558,124821.0,1.0,63.0,4.0,2.0,2.0,3.71


In [224]:
sex_map = {1:'M', 2:'F'}
demo['sex'] = demo['sex'].map(sex_map)
demo

,SEQN,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,6.0,NaN,NaN,4.66
1,109264.0,F,13.0,1.0,NaN,NaN,0.83
2,109265.0,M,2.0,3.0,NaN,NaN,3.06
3,109266.0,F,29.0,6.0,5.0,3.0,5.00
4,109267.0,F,21.0,2.0,4.0,3.0,5.00
...,...,...,...,...,...,...,...
15555,124818.0,M,40.0,4.0,5.0,1.0,3.82
15556,124819.0,M,2.0,4.0,NaN,NaN,0.07
15557,124820.0,F,7.0,3.0,NaN,NaN,1.22
15558,124821.0,M,63.0,4.0,2.0,2.0,3.71


In [225]:
race_map = {1:'Mexican American', 2:'Other Hispanic', 3:'Non-Hispanic White', 4:'Non-Hispanic Black',
            6:'Non-Hispanic Asian', 7:'Other Race - Including Multi-Racial'}
demo['race'] = demo['race'].map(race_map)
demo

,SEQN,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,Non-Hispanic Asian,NaN,NaN,4.66
1,109264.0,F,13.0,Mexican American,NaN,NaN,0.83
2,109265.0,M,2.0,Non-Hispanic White,NaN,NaN,3.06
3,109266.0,F,29.0,Non-Hispanic Asian,5.0,3.0,5.00
4,109267.0,F,21.0,Other Hispanic,4.0,3.0,5.00
...,...,...,...,...,...,...,...
15555,124818.0,M,40.0,Non-Hispanic Black,5.0,1.0,3.82
15556,124819.0,M,2.0,Non-Hispanic Black,NaN,NaN,0.07
15557,124820.0,F,7.0,Non-Hispanic White,NaN,NaN,1.22
15558,124821.0,M,63.0,Non-Hispanic Black,2.0,2.0,3.71


In [226]:
educ_map = {1:'Less than 9th grade', 2:'9-11th grade', 3:'High school graduate/GED or equivalent', 
            4:'Some college or AA degree', 5:'College graduate or above'}
demo['education'] = demo['education'].map(educ_map)
demo

,SEQN,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,Non-Hispanic Asian,NaN,NaN,4.66
1,109264.0,F,13.0,Mexican American,NaN,NaN,0.83
2,109265.0,M,2.0,Non-Hispanic White,NaN,NaN,3.06
3,109266.0,F,29.0,Non-Hispanic Asian,College graduate or above,3.0,5.00
4,109267.0,F,21.0,Other Hispanic,Some college or AA degree,3.0,5.00
...,...,...,...,...,...,...,...
15555,124818.0,M,40.0,Non-Hispanic Black,College graduate or above,1.0,3.82
15556,124819.0,M,2.0,Non-Hispanic Black,NaN,NaN,0.07
15557,124820.0,F,7.0,Non-Hispanic White,NaN,NaN,1.22
15558,124821.0,M,63.0,Non-Hispanic Black,9-11th grade,2.0,3.71


In [227]:
demo['education'].value_counts()

Some college or AA degree                 2941
College graduate or above                 2234
High school graduate/GED or equivalent    2212
9-11th grade                              1028
Less than 9th grade                        715
Name: education, dtype: int64

In [228]:
mar_map = {1:'Married/Living with Partner', 2:'Widowed/Divorced/Separated', 3:'Never married'}
demo['marital_status'] = demo['marital_status'].map(mar_map)
demo

,SEQN,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,Non-Hispanic Asian,NaN,NaN,4.66
1,109264.0,F,13.0,Mexican American,NaN,NaN,0.83
2,109265.0,M,2.0,Non-Hispanic White,NaN,NaN,3.06
3,109266.0,F,29.0,Non-Hispanic Asian,College graduate or above,Never married,5.00
4,109267.0,F,21.0,Other Hispanic,Some college or AA degree,Never married,5.00
...,...,...,...,...,...,...,...
15555,124818.0,M,40.0,Non-Hispanic Black,College graduate or above,Married/Living with Partner,3.82
15556,124819.0,M,2.0,Non-Hispanic Black,NaN,NaN,0.07
15557,124820.0,F,7.0,Non-Hispanic White,NaN,NaN,1.22
15558,124821.0,M,63.0,Non-Hispanic Black,9-11th grade,Widowed/Divorced/Separated,3.71


In [229]:
demo = demo.query('age_year >= 2')

In [11]:
total_intake_1_url = 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DR1TOT.XPT'
total_intake_1_df = pd.read_sas(total_intake_1_url)


/tmp/ipykernel_43/3086884200.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_intake_1_df = pd.read_sas(total_intake_1_url)
/tmp/ipykernel_43/3086884200.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_intake_1_df = pd.read_sas(total_intake_1_url)
/tmp/ipykernel_43/3086884200.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-frag

In [12]:
total_intake_1_df

,SEQN,WTDRD1PP,WTDR2DPP,DR1DRSTZ,DR1EXMER,DRABF,DRDINT,DR1DBIH,DR1DAY,DR1LANG,DR1MRESP,DR1HELP,DBQ095Z,DBD100,DRQSPREP,...,DRD370O,DRD370OQ,DRD370P,DRD370PQ,DRD370Q,DRD370QQ,DRD370R,DRD370RQ,DRD370S,DRD370SQ,DRD370T,DRD370TQ,DRD370U,DRD370UQ,DRD370V
0,109263.0,7.619484e+03,1.780807e+04,1.0,14.0,2.0,2.0,4.000000e+00,6.0,1.0,3.0,12.0,4.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109264.0,8.235896e+03,7.253762e+03,1.0,81.0,2.0,2.0,5.000000e+00,6.0,1.0,1.0,12.0,1.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109265.0,3.353508e+04,3.561201e+04,1.0,88.0,2.0,2.0,1.900000e+01,4.0,1.0,2.0,12.0,4.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109266.0,6.831068e+03,5.988204e+03,1.0,81.0,2.0,2.0,4.000000e+00,7.0,1.0,1.0,12.0,1.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109269.0,7.875707e+03,1.823193e+04,1.0,88.0,2.0,2.0,9.000000e+00,1.0,1.0,2.0,12.0,4.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14295,124818.0,1.546208e+04,1.243740e+04,1.0,49.0,2.0,2.0,6.000000e+00,7.0,1.0,1.0,12.0,4.0,NaN,3.0,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
14296,124819.0,4.091615e+03,4.100758e+03,1.0,14.0,2.0,2.0,2.200000e+01,4.0,1.0,2.0,12.0,1.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14297,124820.0,3.435783e+04,3.505186e+04,1.0,86.0,2.0,2.0,1.800000e+01,4.0,1.0,2.0,1.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14298,124821.0,3.043724e+03,5.397605e-79,1.0,81.0,2.0,1.0,5.397605e-79,7.0,1.0,1.0,12.0,1.0,2.0,3.0,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0


In [13]:
total_intake_1_df = total_intake_1_df[total_intake_1_df.DR1DRSTZ == 1] #drops all the rows with incomplete dietary records

In [14]:
total_intake_1_df.shape

(12392, 168)

In [15]:
print(total_intake_1_df.columns.tolist())

['SEQN', 'WTDRD1PP', 'WTDR2DPP', 'DR1DRSTZ', 'DR1EXMER', 'DRABF', 'DRDINT', 'DR1DBIH', 'DR1DAY', 'DR1LANG', 'DR1MRESP', 'DR1HELP', 'DBQ095Z', 'DBD100', 'DRQSPREP', 'DR1STY', 'DR1SKY', 'DRQSDIET', 'DRQSDT1', 'DRQSDT2', 'DRQSDT3', 'DRQSDT4', 'DRQSDT5', 'DRQSDT6', 'DRQSDT7', 'DRQSDT8', 'DRQSDT9', 'DRQSDT10', 'DRQSDT11', 'DRQSDT12', 'DRQSDT91', 'DR1TNUMF', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR', 'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL', 'DR1TATOC', 'DR1TATOA', 'DR1TRET', 'DR1TVARA', 'DR1TACAR', 'DR1TBCAR', 'DR1TCRYP', 'DR1TLYCO', 'DR1TLZ', 'DR1TVB1', 'DR1TVB2', 'DR1TNIAC', 'DR1TVB6', 'DR1TFOLA', 'DR1TFA', 'DR1TFF', 'DR1TFDFE', 'DR1TCHL', 'DR1TVB12', 'DR1TB12A', 'DR1TVC', 'DR1TVD', 'DR1TVK', 'DR1TCALC', 'DR1TPHOS', 'DR1TMAGN', 'DR1TIRON', 'DR1TZINC', 'DR1TCOPP', 'DR1TSODI', 'DR1TPOTA', 'DR1TSELE', 'DR1TCAFF', 'DR1TTHEO', 'DR1TALCO', 'DR1TMOIS', 'DR1TS040', 'DR1TS060', 'DR1TS080', 'DR1TS100', 'DR1TS120', 'DR1TS140', 'DR1TS160', 'DR1TS180', 'DR1TM161', 'D

In [16]:
cols_to_keep = ['SEQN', 'DRDINT', 'DR1TATOC', 'DR1TVARA', 'DR1TVB1', 'DR1TVB2',
                'DR1TNIAC', 'DR1TVB6', 'DR1TFOLA','DR1TCHL', 'DR1TVB12', 'DR1TVC', 'DR1TVD', 'DR1TVK', 
                'DR1TCALC', 'DR1TPHOS', 'DR1TMAGN', 'DR1TIRON', 'DR1TZINC', 'DR1TSODI', 'DR1TPOTA'] #'WTDRD1PP', 'WTDR2DPP',
nutr_1_clean = total_intake_1_df[cols_to_keep]
nutr_1_clean

,SEQN,DRDINT,DR1TATOC,DR1TVARA,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TSODI,DR1TPOTA
0,109263.0,2.0,6.91,373.0,2.121,1.322,19.786,1.272,433.0,171.3,2.89,125.6,4.0,65.2,817.0,933.0,175.0,13.73,8.06,2456.0,1401.0
1,109264.0,2.0,4.63,694.0,1.017,1.090,24.607,2.351,136.0,173.2,1.63,15.2,0.1,95.6,399.0,935.0,167.0,8.46,3.73,1703.0,1220.0
2,109265.0,2.0,3.26,401.0,0.983,1.216,18.123,1.431,203.0,168.2,2.68,83.8,2.8,46.2,1056.0,1158.0,200.0,6.02,6.08,2221.0,2172.0
3,109266.0,2.0,12.48,222.0,0.945,1.351,8.461,0.772,245.0,252.0,1.65,63.5,1.6,75.5,991.0,1182.0,300.0,12.07,6.64,2001.0,2014.0
4,109269.0,2.0,3.78,164.0,0.908,0.686,8.974,0.363,193.0,82.2,1.07,156.6,0.1,93.8,372.0,604.0,85.0,6.90,3.06,2162.0,1210.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14294,124817.0,1.0,4.46,530.0,0.402,1.164,4.814,0.691,183.0,563.2,3.71,75.5,4.2,19.4,355.0,704.0,142.0,7.86,8.72,1129.0,930.0
14295,124818.0,2.0,14.70,743.0,1.875,1.779,18.603,0.647,531.0,144.8,1.17,89.3,2.3,103.5,480.0,883.0,186.0,15.08,6.50,7742.0,1206.0
14296,124819.0,2.0,3.52,660.0,1.416,1.921,15.602,1.421,233.0,236.8,5.20,93.4,13.0,17.4,1584.0,1545.0,201.0,8.32,8.00,2870.0,2356.0
14297,124820.0,2.0,4.40,698.0,1.563,1.761,19.505,2.002,486.0,115.0,6.00,28.2,7.0,41.4,1384.0,1034.0,161.0,16.36,14.09,2344.0,1212.0


In [17]:
nutr_1_clean = nutr_1_clean.rename({'SEQN': 'SEQN',
                              'WTDRD1PP':'day_one_sample_w', 
                              'WTDR2DPP':'two_day_sample_w',
                              'DRDINT':'num_days_intake',
                              'DR1TATOC':'d1_vit_e',
                              'DR1TATOA':'d1_vit_e_added', 
                              'DR1TVARA':'d1_vit_a',
                              'DR1TVB1':'d1_thiamin_b1',
                              'DR1TVB2':'d1_riboflavin_b2',
                              'DR1TNIAC': 'd1_niacin',
                              'DR1TVB6':'d1_vit_b6',
                              'DR1TFOLA':'d1_folate',
                              'DR1TCHL':'d1_choline',
                              'DR1TVB12':'d1_vit_b12',
                              'DR1TVC':'d1_vit_c',
                              'DR1TVD':'d1_vit_d',
                              'DR1TVK':'d1_vit_k',
                              'DR1TCALC':'d1_calcium',
                              'DR1TPHOS':'d1_phosphorus',
                              'DR1TMAGN':'d1_magnesium',
                              'DR1TIRON':'d1_iron',
                              'DR1TZINC':'d1_zinc',
                              'DR1TSODI':'d1_sodium',
                              'DR1TPOTA':'d1_potassium'},
                            axis=1)
nutr_1_clean

,SEQN,num_days_intake,d1_vit_e,d1_vit_a,d1_thiamin_b1,d1_riboflavin_b2,d1_niacin,d1_vit_b6,d1_folate,d1_choline,d1_vit_b12,d1_vit_c,d1_vit_d,d1_vit_k,d1_calcium,d1_phosphorus,d1_magnesium,d1_iron,d1_zinc,d1_sodium,d1_potassium
0,109263.0,2.0,6.91,373.0,2.121,1.322,19.786,1.272,433.0,171.3,2.89,125.6,4.0,65.2,817.0,933.0,175.0,13.73,8.06,2456.0,1401.0
1,109264.0,2.0,4.63,694.0,1.017,1.090,24.607,2.351,136.0,173.2,1.63,15.2,0.1,95.6,399.0,935.0,167.0,8.46,3.73,1703.0,1220.0
2,109265.0,2.0,3.26,401.0,0.983,1.216,18.123,1.431,203.0,168.2,2.68,83.8,2.8,46.2,1056.0,1158.0,200.0,6.02,6.08,2221.0,2172.0
3,109266.0,2.0,12.48,222.0,0.945,1.351,8.461,0.772,245.0,252.0,1.65,63.5,1.6,75.5,991.0,1182.0,300.0,12.07,6.64,2001.0,2014.0
4,109269.0,2.0,3.78,164.0,0.908,0.686,8.974,0.363,193.0,82.2,1.07,156.6,0.1,93.8,372.0,604.0,85.0,6.90,3.06,2162.0,1210.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14294,124817.0,1.0,4.46,530.0,0.402,1.164,4.814,0.691,183.0,563.2,3.71,75.5,4.2,19.4,355.0,704.0,142.0,7.86,8.72,1129.0,930.0
14295,124818.0,2.0,14.70,743.0,1.875,1.779,18.603,0.647,531.0,144.8,1.17,89.3,2.3,103.5,480.0,883.0,186.0,15.08,6.50,7742.0,1206.0
14296,124819.0,2.0,3.52,660.0,1.416,1.921,15.602,1.421,233.0,236.8,5.20,93.4,13.0,17.4,1584.0,1545.0,201.0,8.32,8.00,2870.0,2356.0
14297,124820.0,2.0,4.40,698.0,1.563,1.761,19.505,2.002,486.0,115.0,6.00,28.2,7.0,41.4,1384.0,1034.0,161.0,16.36,14.09,2344.0,1212.0


In [18]:
total_intake_2_url = 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DR2TOT.XPT'
total_intake_2_df = pd.read_sas(total_intake_2_url)
total_intake_2_df = total_intake_2_df[total_intake_2_df.DR2DRSTZ == 1] #drops all the rows with incomplete dietary records
cols_to_keep = ['SEQN', 'DRDINT', 'DR2TATOC', 'DR2TVARA', 'DR2TVB1', 'DR2TVB2',
                'DR2TNIAC', 'DR2TVB6', 'DR2TFOLA','DR2TCHL', 'DR2TVB12', 'DR2TVC', 'DR2TVD', 'DR2TVK', 
                'DR2TCALC', 'DR2TPHOS', 'DR2TMAGN', 'DR2TIRON', 'DR2TZINC', 'DR2TSODI', 'DR2TPOTA'] #'WTDRD1PP', 'WTDR2DPP',
nutr_2_clean = total_intake_2_df[cols_to_keep]
nutr_2_clean = nutr_2_clean.rename({'SEQN': 'SEQN',
                              'WTDRD1PP':'day_one_sample_w', #interviewed only or MEC and interviewed
                              'WTDR2DPP':'two_day_sample_w', # 1 male, 2 female
                              'DRDINT':'num_days_intake',
                              'DR2TATOC':'d2_vit_e',
                              'DR2TVARA':'d2_vit_a',
                              'DR2TVB1':'d2_thiamin_b1',
                              'DR2TVB2':'d2_riboflavin_b2',
                              'DR2TNIAC': 'd2_niacin',
                              'DR2TVB6':'d2_vit_b6',
                              'DR2TFOLA':'d2_folate',
                              'DR2TCHL':'d2_choline',
                              'DR2TVB12':'d2_vit_b12',
                              'DR2TVC':'d2_vit_c',
                              'DR2TVD':'d2_vit_d',
                              'DR2TVK':'d2_vit_k',
                              'DR2TCALC':'d2_calcium',
                              'DR2TPHOS':'d2_phosphorus',
                              'DR2TMAGN':'d2_magnesium',
                              'DR2TIRON':'d2_iron',
                              'DR2TZINC':'d2_zinc',
                              'DR2TSODI':'d2_sodium',
                              'DR2TPOTA':'d2_potassium'},
                            axis=1)

In [19]:
nutr_2_clean

,SEQN,num_days_intake,d2_vit_e,d2_vit_a,d2_thiamin_b1,d2_riboflavin_b2,d2_niacin,d2_vit_b6,d2_folate,d2_choline,d2_vit_b12,d2_vit_c,d2_vit_d,d2_vit_k,d2_calcium,d2_phosphorus,d2_magnesium,d2_iron,d2_zinc,d2_sodium,d2_potassium
0,109263.0,2.0,2.38,236.0,1.271,0.708,11.091,1.067,411.0,107.0,1.80,209.3,7.5,5.4,1236.0,584.0,145.0,7.50,4.73,1390.0,1722.0
1,109264.0,2.0,5.57,1551.0,3.383,2.664,47.689,4.834,1182.0,223.1,16.13,74.6,7.2,141.9,744.0,1176.0,226.0,48.03,29.13,2687.0,1542.0
2,109265.0,2.0,5.62,865.0,1.386,1.892,14.782,1.435,275.0,227.1,5.94,48.7,9.4,68.8,1168.0,1226.0,192.0,8.44,8.56,1761.0,2386.0
3,109266.0,2.0,9.62,618.0,1.641,1.937,16.597,1.890,319.0,194.3,3.21,95.5,6.5,106.9,1105.0,1311.0,384.0,11.03,8.63,3439.0,2794.0
4,109269.0,2.0,4.37,396.0,0.259,0.937,2.025,0.300,121.0,454.3,1.45,99.5,5.2,16.7,345.0,492.0,66.0,4.82,3.02,845.0,953.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14291,124814.0,2.0,4.94,352.0,1.039,1.068,12.158,0.630,218.0,224.2,1.34,217.3,3.0,51.6,746.0,812.0,137.0,8.90,4.73,2149.0,1064.0
14292,124815.0,2.0,2.68,39.0,1.213,1.142,21.221,1.524,203.0,371.0,4.72,125.1,1.5,8.3,268.0,903.0,175.0,5.58,5.85,1541.0,1817.0
14295,124818.0,2.0,9.12,1419.0,1.889,1.811,17.369,1.779,320.0,392.3,6.01,64.2,4.9,90.6,1454.0,1697.0,246.0,13.25,15.85,6120.0,2774.0
14296,124819.0,2.0,3.54,73.0,0.653,0.409,9.724,1.073,83.0,103.9,1.84,139.7,0.1,24.6,429.0,417.0,108.0,4.89,4.17,1398.0,2004.0


In [20]:
nutr = pd.merge(nutr_1_clean, nutr_2_clean,
                        on = ['SEQN'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')
nutr

,SEQN,num_days_intake_x,d1_vit_e,d1_vit_a,d1_thiamin_b1,d1_riboflavin_b2,d1_niacin,d1_vit_b6,d1_folate,d1_choline,d1_vit_b12,d1_vit_c,d1_vit_d,d1_vit_k,d1_calcium,...,d2_vit_b6,d2_folate,d2_choline,d2_vit_b12,d2_vit_c,d2_vit_d,d2_vit_k,d2_calcium,d2_phosphorus,d2_magnesium,d2_iron,d2_zinc,d2_sodium,d2_potassium,matched
0,109263.0,2.0,6.91,373.0,2.121,1.322,19.786,1.272,433.0,171.3,2.89,125.6,4.0,65.2,817.0,...,1.067,411.0,107.0,1.80,209.3,7.5,5.4,1236.0,584.0,145.0,7.50,4.73,1390.0,1722.0,both
1,109264.0,2.0,4.63,694.0,1.017,1.090,24.607,2.351,136.0,173.2,1.63,15.2,0.1,95.6,399.0,...,4.834,1182.0,223.1,16.13,74.6,7.2,141.9,744.0,1176.0,226.0,48.03,29.13,2687.0,1542.0,both
2,109265.0,2.0,3.26,401.0,0.983,1.216,18.123,1.431,203.0,168.2,2.68,83.8,2.8,46.2,1056.0,...,1.435,275.0,227.1,5.94,48.7,9.4,68.8,1168.0,1226.0,192.0,8.44,8.56,1761.0,2386.0,both
3,109266.0,2.0,12.48,222.0,0.945,1.351,8.461,0.772,245.0,252.0,1.65,63.5,1.6,75.5,991.0,...,1.890,319.0,194.3,3.21,95.5,6.5,106.9,1105.0,1311.0,384.0,11.03,8.63,3439.0,2794.0,both
4,109269.0,2.0,3.78,164.0,0.908,0.686,8.974,0.363,193.0,82.2,1.07,156.6,0.1,93.8,372.0,...,0.300,121.0,454.3,1.45,99.5,5.2,16.7,345.0,492.0,66.0,4.82,3.02,845.0,953.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,118617.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.809,234.0,147.3,3.71,131.9,5.4,63.4,1276.0,778.0,189.0,25.78,10.42,296.0,2138.0,right_only
12405,120172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.582,138.0,219.6,2.34,82.8,16.8,75.6,798.0,534.0,54.0,17.10,7.14,288.0,1020.0,right_only
12406,120907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.525,121.0,75.2,1.85,51.4,2.3,10.6,285.0,374.0,81.0,4.77,3.77,830.0,958.0,right_only
12407,122957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.124,163.0,156.8,2.31,29.4,3.5,25.6,792.0,1030.0,201.0,7.26,6.24,2009.0,1718.0,right_only


In [21]:
nutr['matched'].value_counts()

both          10610
left_only      1782
right_only       17
Name: matched, dtype: int64

In [22]:
nutr['avg_vit_e'] = nutr[['d1_vit_e', 'd2_vit_e']].mean(axis=1)
nutr['avg_vit_a'] = nutr[['d1_vit_a', 'd2_vit_a']].mean(axis=1)
nutr['avg_thiamin_b1'] = nutr[['d1_thiamin_b1', 'd2_thiamin_b1']].mean(axis=1)
nutr['avg_riboflavin_b2'] = nutr[['d1_riboflavin_b2', 'd2_riboflavin_b2']].mean(axis=1)
nutr['avg_niacin'] = nutr[['d1_niacin', 'd2_niacin']].mean(axis=1)
nutr['avg_vit_b6'] = nutr[['d1_vit_b6', 'd2_vit_b6']].mean(axis=1)
nutr['avg_folate'] = nutr[['d1_folate', 'd2_folate']].mean(axis=1)
nutr['avg_choline'] = nutr[['d1_choline', 'd2_choline']].mean(axis=1)
nutr['avg_vit_b12'] = nutr[['d1_vit_b12', 'd2_vit_b12']].mean(axis=1)
nutr['avg_vit_c'] = nutr[['d1_vit_c', 'd2_vit_c']].mean(axis=1)
nutr['avg_vit_d'] = nutr[['d1_vit_d', 'd2_vit_d']].mean(axis=1)
nutr['avg_vit_k'] = nutr[['d1_vit_k', 'd2_vit_k']].mean(axis=1)
nutr['avg_calcium'] = nutr[['d1_calcium', 'd2_calcium']].mean(axis=1)
nutr['avg_phosphorus'] = nutr[['d1_phosphorus', 'd2_phosphorus']].mean(axis=1)
nutr['avg_magnesium'] = nutr[['d1_magnesium', 'd2_magnesium']].mean(axis=1)
nutr['avg_iron'] = nutr[['d1_iron', 'd2_iron']].mean(axis=1)
nutr['avg_zinc'] = nutr[['d1_zinc', 'd2_zinc']].mean(axis=1)
nutr['avg_sodium'] = nutr[['d1_sodium', 'd2_sodium']].mean(axis=1)
nutr['avg_potassium'] = nutr[['d1_potassium', 'd2_potassium']].mean(axis=1)
nutr

,SEQN,num_days_intake_x,d1_vit_e,d1_vit_a,d1_thiamin_b1,d1_riboflavin_b2,d1_niacin,d1_vit_b6,d1_folate,d1_choline,d1_vit_b12,d1_vit_c,d1_vit_d,d1_vit_k,d1_calcium,...,avg_niacin,avg_vit_b6,avg_folate,avg_choline,avg_vit_b12,avg_vit_c,avg_vit_d,avg_vit_k,avg_calcium,avg_phosphorus,avg_magnesium,avg_iron,avg_zinc,avg_sodium,avg_potassium
0,109263.0,2.0,6.91,373.0,2.121,1.322,19.786,1.272,433.0,171.3,2.89,125.6,4.0,65.2,817.0,...,15.4385,1.1695,422.0,139.15,2.345,167.45,5.75,35.30,1026.5,758.5,160.0,10.615,6.395,1923.0,1561.5
1,109264.0,2.0,4.63,694.0,1.017,1.090,24.607,2.351,136.0,173.2,1.63,15.2,0.1,95.6,399.0,...,36.1480,3.5925,659.0,198.15,8.880,44.90,3.65,118.75,571.5,1055.5,196.5,28.245,16.430,2195.0,1381.0
2,109265.0,2.0,3.26,401.0,0.983,1.216,18.123,1.431,203.0,168.2,2.68,83.8,2.8,46.2,1056.0,...,16.4525,1.4330,239.0,197.65,4.310,66.25,6.10,57.50,1112.0,1192.0,196.0,7.230,7.320,1991.0,2279.0
3,109266.0,2.0,12.48,222.0,0.945,1.351,8.461,0.772,245.0,252.0,1.65,63.5,1.6,75.5,991.0,...,12.5290,1.3310,282.0,223.15,2.430,79.50,4.05,91.20,1048.0,1246.5,342.0,11.550,7.635,2720.0,2404.0
4,109269.0,2.0,3.78,164.0,0.908,0.686,8.974,0.363,193.0,82.2,1.07,156.6,0.1,93.8,372.0,...,5.4995,0.3315,157.0,268.25,1.260,128.05,2.65,55.25,358.5,548.0,75.5,5.860,3.040,1503.5,1081.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,118617.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.6300,1.8090,234.0,147.30,3.710,131.90,5.40,63.40,1276.0,778.0,189.0,25.780,10.420,296.0,2138.0
12405,120172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0080,0.5820,138.0,219.60,2.340,82.80,16.80,75.60,798.0,534.0,54.0,17.100,7.140,288.0,1020.0
12406,120907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.9740,0.5250,121.0,75.20,1.850,51.40,2.30,10.60,285.0,374.0,81.0,4.770,3.770,830.0,958.0
12407,122957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.4810,1.1240,163.0,156.80,2.310,29.40,3.50,25.60,792.0,1030.0,201.0,7.260,6.240,2009.0,1718.0


In [230]:
cols_to_keep = ['SEQN', 'avg_vit_e', 'avg_vit_a', 'avg_thiamin_b1', 'avg_riboflavin_b2', 'avg_niacin',
                'avg_vit_b6', 'avg_folate', 'avg_choline','avg_vit_b12', 'avg_vit_c', 'avg_vit_d', 'avg_vit_k', 'avg_calcium', 
                'avg_phosphorus', 'avg_magnesium', 'avg_iron', 'avg_zinc', 'avg_sodium', 'avg_potassium'] #'WTDRD1PP', 'WTDR2DPP',
nutr = nutr[cols_to_keep]
nutr

,SEQN,avg_vit_e,avg_vit_a,avg_thiamin_b1,avg_riboflavin_b2,avg_niacin,avg_vit_b6,avg_folate,avg_choline,avg_vit_b12,avg_vit_c,avg_vit_d,avg_vit_k,avg_calcium,avg_phosphorus,avg_magnesium,avg_iron,avg_zinc,avg_sodium,avg_potassium
0,109263.0,4.645,304.5,1.6960,1.0150,15.4385,1.1695,422.0,139.15,2.345,167.45,5.75,35.30,1026.5,758.5,160.0,10.615,6.395,1923.0,1561.5
1,109264.0,5.100,1122.5,2.2000,1.8770,36.1480,3.5925,659.0,198.15,8.880,44.90,3.65,118.75,571.5,1055.5,196.5,28.245,16.430,2195.0,1381.0
2,109265.0,4.440,633.0,1.1845,1.5540,16.4525,1.4330,239.0,197.65,4.310,66.25,6.10,57.50,1112.0,1192.0,196.0,7.230,7.320,1991.0,2279.0
3,109266.0,11.050,420.0,1.2930,1.6440,12.5290,1.3310,282.0,223.15,2.430,79.50,4.05,91.20,1048.0,1246.5,342.0,11.550,7.635,2720.0,2404.0
4,109269.0,4.075,280.0,0.5835,0.8115,5.4995,0.3315,157.0,268.25,1.260,128.05,2.65,55.25,358.5,548.0,75.5,5.860,3.040,1503.5,1081.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,118617.0,7.330,1219.0,1.0880,1.5340,14.6300,1.8090,234.0,147.30,3.710,131.90,5.40,63.40,1276.0,778.0,189.0,25.780,10.420,296.0,2138.0
12405,120172.0,12.960,858.0,0.9480,1.4280,10.0080,0.5820,138.0,219.60,2.340,82.80,16.80,75.60,798.0,534.0,54.0,17.100,7.140,288.0,1020.0
12406,120907.0,2.210,130.0,0.4220,0.4940,3.9740,0.5250,121.0,75.20,1.850,51.40,2.30,10.60,285.0,374.0,81.0,4.770,3.770,830.0,958.0
12407,122957.0,4.170,266.0,0.9370,1.1750,13.4810,1.1240,163.0,156.80,2.310,29.40,3.50,25.60,792.0,1030.0,201.0,7.260,6.240,2009.0,1718.0


In [231]:
pop_nutr = pd.merge(nutr, demo,
                        on = ['SEQN'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')
pop_nutr['matched'].value_counts()

both          11727
right_only     2766
left_only       682
Name: matched, dtype: int64

In [232]:
pop_nutr = pop_nutr.query("matched == 'both'")[cols_to_keep]
pop_nutr

,SEQN,avg_vit_e,avg_vit_a,avg_thiamin_b1,avg_riboflavin_b2,avg_niacin,avg_vit_b6,avg_folate,avg_choline,avg_vit_b12,avg_vit_c,avg_vit_d,avg_vit_k,avg_calcium,avg_phosphorus,avg_magnesium,avg_iron,avg_zinc,avg_sodium,avg_potassium
0,109263.0,4.645,304.5,1.6960,1.0150,15.4385,1.1695,422.0,139.15,2.345,167.45,5.75,35.30,1026.5,758.5,160.0,10.615,6.395,1923.0,1561.5
1,109264.0,5.100,1122.5,2.2000,1.8770,36.1480,3.5925,659.0,198.15,8.880,44.90,3.65,118.75,571.5,1055.5,196.5,28.245,16.430,2195.0,1381.0
2,109265.0,4.440,633.0,1.1845,1.5540,16.4525,1.4330,239.0,197.65,4.310,66.25,6.10,57.50,1112.0,1192.0,196.0,7.230,7.320,1991.0,2279.0
3,109266.0,11.050,420.0,1.2930,1.6440,12.5290,1.3310,282.0,223.15,2.430,79.50,4.05,91.20,1048.0,1246.5,342.0,11.550,7.635,2720.0,2404.0
4,109269.0,4.075,280.0,0.5835,0.8115,5.4995,0.3315,157.0,268.25,1.260,128.05,2.65,55.25,358.5,548.0,75.5,5.860,3.040,1503.5,1081.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,112157.0,6.410,555.0,2.7330,2.7110,43.5520,4.3110,560.0,215.50,9.870,43.80,8.20,38.20,785.0,1006.0,250.0,29.710,8.140,2726.0,2179.0
12399,115293.0,4.320,410.0,1.1100,2.1990,13.5170,0.8230,236.0,222.30,5.420,14.60,15.00,51.30,1459.0,1343.0,198.0,5.940,6.550,1677.0,2458.0
12403,117889.0,4.460,463.0,1.0850,1.4080,13.1980,1.1280,257.0,232.00,2.760,64.80,3.80,30.20,897.0,938.0,170.0,10.680,7.550,1721.0,1469.0
12407,122957.0,4.170,266.0,0.9370,1.1750,13.4810,1.1240,163.0,156.80,2.310,29.40,3.50,25.60,792.0,1030.0,201.0,7.260,6.240,2009.0,1718.0


In [234]:
pop_nutr['avg_vit_d'] = pop_nutr['avg_vit_d'] * 40
pop_nutr

,SEQN,avg_vit_e,avg_vit_a,avg_thiamin_b1,avg_riboflavin_b2,avg_niacin,avg_vit_b6,avg_folate,avg_choline,avg_vit_b12,avg_vit_c,avg_vit_d,avg_vit_k,avg_calcium,avg_phosphorus,avg_magnesium,avg_iron,avg_zinc,avg_sodium,avg_potassium
0,109263.0,4.645,304.5,1.6960,1.0150,15.4385,1.1695,422.0,139.15,2.345,167.45,230.0,35.30,1026.5,758.5,160.0,10.615,6.395,1923.0,1561.5
1,109264.0,5.100,1122.5,2.2000,1.8770,36.1480,3.5925,659.0,198.15,8.880,44.90,146.0,118.75,571.5,1055.5,196.5,28.245,16.430,2195.0,1381.0
2,109265.0,4.440,633.0,1.1845,1.5540,16.4525,1.4330,239.0,197.65,4.310,66.25,244.0,57.50,1112.0,1192.0,196.0,7.230,7.320,1991.0,2279.0
3,109266.0,11.050,420.0,1.2930,1.6440,12.5290,1.3310,282.0,223.15,2.430,79.50,162.0,91.20,1048.0,1246.5,342.0,11.550,7.635,2720.0,2404.0
4,109269.0,4.075,280.0,0.5835,0.8115,5.4995,0.3315,157.0,268.25,1.260,128.05,106.0,55.25,358.5,548.0,75.5,5.860,3.040,1503.5,1081.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,112157.0,6.410,555.0,2.7330,2.7110,43.5520,4.3110,560.0,215.50,9.870,43.80,328.0,38.20,785.0,1006.0,250.0,29.710,8.140,2726.0,2179.0
12399,115293.0,4.320,410.0,1.1100,2.1990,13.5170,0.8230,236.0,222.30,5.420,14.60,600.0,51.30,1459.0,1343.0,198.0,5.940,6.550,1677.0,2458.0
12403,117889.0,4.460,463.0,1.0850,1.4080,13.1980,1.1280,257.0,232.00,2.760,64.80,152.0,30.20,897.0,938.0,170.0,10.680,7.550,1721.0,1469.0
12407,122957.0,4.170,266.0,0.9370,1.1750,13.4810,1.1240,163.0,156.80,2.310,29.40,140.0,25.60,792.0,1030.0,201.0,7.260,6.240,2009.0,1718.0


In [233]:
demo = pd.merge(nutr, demo,
                        on = ['SEQN'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')
cols_to_keep = ['SEQN', 'sex','age_year', 'race', 'education', 'marital_status', 'income_pov_rat']
demo = demo.query("matched == 'both'")[cols_to_keep]
demo

,SEQN,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,Non-Hispanic Asian,NaN,NaN,4.66
1,109264.0,F,13.0,Mexican American,NaN,NaN,0.83
2,109265.0,M,2.0,Non-Hispanic White,NaN,NaN,3.06
3,109266.0,F,29.0,Non-Hispanic Asian,College graduate or above,Never married,5.00
4,109269.0,M,2.0,Other Hispanic,NaN,NaN,0.96
...,...,...,...,...,...,...,...
12394,112157.0,M,3.0,Mexican American,NaN,NaN,0.48
12399,115293.0,F,2.0,Non-Hispanic White,NaN,NaN,0.99
12403,117889.0,F,2.0,Other Hispanic,NaN,NaN,0.85
12407,122957.0,F,2.0,Non-Hispanic Black,NaN,NaN,1.47


In [74]:
#rec = pd.DataFrame(columns=['age_lower', 'age_upper', 'sex', 'vit_a', 'vit_e', 'vit_d', 'vit_c', 'thiamin_b1', 
                           #'riboflavin_b2', 'niacin', 'vit_b6', 'vit_b12', 'choline', 'vit_k', 'folate'])

rec_dict = {'age_lower':[2,4,9,14,19,31,51,4,9,14,19,31,51],
            'age_upper':[3,8,13,18,30,50,150,8,13,18,30,50,150],
            'sex':['M/F','F','F','F','F','F','F','M','M','M','M','M','M'],
            'vit_a':[300, 400, 600, 700, 700, 700, 700, 400, 600, 900, 900, 900, 900], 
            'vit_e':[6, 7, 11, 15, 15, 15, 15, 7, 11, 15, 15, 15, 15], 
            'vit_d':[600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600], 
            'vit_c':[15, 25, 45, 65, 75, 75, 75, 25, 45, 75, 90, 90, 90], 
            'thiamin_b1':[0.5, 0.6, 0.9, 1.0, 1.1, 1.1, 1.1, 0.6, 0.9, 1.2, 1.2, 1.2, 1.2], 
            'riboflavin_b2':[0.5, 0.6, 0.9, 1.0, 1.1, 1.1, 1.1, 0.6, 0.9, 1.3, 1.3, 1.3, 1.3], 
            'niacin':[6, 8, 12, 14, 14, 14, 14, 8, 12, 16, 16, 16, 16], 
            'vit_b6':[0.5, 0.6, 1.0, 1.2, 1.3, 1.3, 1.5, 0.6, 1.0, 1.3, 1.3, 1.3, 1.7], 
            'vit_b12':[0.9, 1.2, 1.8, 2.4, 2.4, 2.4, 2.4, 1.2, 1.8, 2.4, 2.4, 2.4, 2.4], 
            'choline':[200, 250, 375, 400, 425, 425, 425, 250, 375, 550, 550, 550, 550], 
            'vit_k':[30, 55, 60, 75, 90, 90, 90, 55, 60, 75, 120, 120, 120], 
            'folate':[150, 200, 300, 400, 400, 400, 400, 200, 300, 400, 400, 400, 400],
            'calcium':[700, 1000, 1300, 1300, 1000, 1000, 1200, 1000, 1300, 1300, 1000, 1000, 1000],
            'iron':[7, 10, 8, 15, 18, 18, 8, 10, 8, 11, 8, 8, 8],
            'magnesium':[80, 130, 240, 360, 310, 320, 320, 130, 240, 410, 400, 420, 420],
            'phosphorus':[460, 500, 1250, 1250, 700, 700, 700, 500, 1250, 1250, 700, 700, 700],
            'potassium':[2000, 2300, 2300, 2300, 2600, 2600, 2600, 2300, 2500, 3000, 3400, 3400, 3400],
            'sodium':[1200, 1500, 1800, 2300, 2300, 2300, 2300, 1500, 1800, 2300, 2300, 2300, 2300],
            'zinc':[3, 5, 8, 9, 8, 8, 8, 5, 8, 11, 11, 11, 11]
       }
  
rec = pd.DataFrame(rec_dict)
  
rec

,age_lower,age_upper,sex,vit_a,vit_e,vit_d,vit_c,thiamin_b1,riboflavin_b2,niacin,vit_b6,vit_b12,choline,vit_k,folate,calcium,iron,magnesium,phosphorus,potassium,sodium,zinc
0,2,3,M/F,300,6,600,15,0.5,0.5,6,0.5,0.9,200,30,150,700,7,80,460,2000,1200,3
1,4,8,F,400,7,600,25,0.6,0.6,8,0.6,1.2,250,55,200,1000,10,130,500,2300,1500,5
2,9,13,F,600,11,600,45,0.9,0.9,12,1.0,1.8,375,60,300,1300,8,240,1250,2300,1800,8
3,14,18,F,700,15,600,65,1.0,1.0,14,1.2,2.4,400,75,400,1300,15,360,1250,2300,2300,9
4,19,30,F,700,15,600,75,1.1,1.1,14,1.3,2.4,425,90,400,1000,18,310,700,2600,2300,8
5,31,50,F,700,15,600,75,1.1,1.1,14,1.3,2.4,425,90,400,1000,18,320,700,2600,2300,8
6,51,150,F,700,15,600,75,1.1,1.1,14,1.5,2.4,425,90,400,1200,8,320,700,2600,2300,8
7,4,8,M,400,7,600,25,0.6,0.6,8,0.6,1.2,250,55,200,1000,10,130,500,2300,1500,5
8,9,13,M,600,11,600,45,0.9,0.9,12,1.0,1.8,375,60,300,1300,8,240,1250,2500,1800,8
9,14,18,M,900,15,600,75,1.2,1.3,16,1.3,2.4,550,75,400,1300,11,410,1250,3000,2300,11


In [245]:
units_dict = {'vit_a':['mcg RAE'], 
            'vit_e':['mg AT'], 
            'vit_d':['IU'], 
            'vit_c':['mg'], 
            'thiamin_b1':['mg'], 
            'riboflavin_b2':['mg'], 
            'niacin':['mg'], 
            'vit_b6':['mg'], 
            'vit_b12':['mcg'], 
            'choline':['mg'], 
            'vit_k':['mcg'], 
            'folate':['mcg DFE'],
            'calcium':['mg'],
            'iron':['mg'],
            'magnesium':['mg'],
            'phosphorus':['mg'],
            'potassium':['mg'],
            'sodium':['mg'],
            'zinc':['mg']
       }
  
units = pd.DataFrame(units_dict)
  
units = units.T
units['units'] = units[0]
units = units.drop([0], axis = 1)
units

,units
vit_a,mcg RAE
vit_e,mg AT
vit_d,IU
vit_c,mg
thiamin_b1,mg
riboflavin_b2,mg
niacin,mg
vit_b6,mg
vit_b12,mcg
choline,mg


In [187]:
os.chdir('..')
usda_token = os.environ['USDA_KEY']
os.chdir('app')

In [26]:
useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']

In [27]:
headers = {'User-Agent' : useragent,
          'From': 'bwl5cd@virginia.edu'}

In [28]:
#https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Foundation&pageSize=50&pageNumber=1
#/v1/foods/list
datatype = 'Foundation'
pagesize=200
pagenum=1
root = 'https://api.nal.usda.gov'
start_date = '2022-10-6'
asteroids_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={pagenum}&api_key={usda_token}'.format(api_key=usda_token)
r = requests.get(root + asteroids_endpoint,
                headers = headers)
r

<Response [200]>

In [102]:
myjson = json.loads(r.text)
#myjson
#pd.json_normalize(myjson)
pd.json_normalize(myjson, "foodNutrients", ['fdcId'])
#pd.json_normalize(myjson, "fdcId", ["description", "dataType", "foodNutrients"])
#pd.json_normalize(myjson, record_path=['foodNutrients'])

,fdcId,description,dataType,publicationDate,ndbNumber,foodNutrients
0,2262074,"Almond butter, creamy",Foundation,2022-04-28,12195,"[{'number': '717', 'name': 'Daidzin', 'amount'..."
1,2257045,"Almond milk, unsweetened, plain, refrigerated",Foundation,2022-04-28,100276,"[{'number': '404', 'name': 'Thiamin', 'amount'..."
2,1999631,"Almond milk, unsweetened, plain, shelf stable",Foundation,2021-10-28,14091,"[{'number': '631', 'name': 'PUFA 22:5 n-3 (DPA..."
3,2003590,"Apple juice, with added vitamin C, from concen...",Foundation,2021-10-28,9400,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
4,1750340,"Apples, fuji, with skin, raw",Foundation,2020-10-30,9504,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
...,...,...,...,...,...,...
195,1999630,"Soy milk, unsweetened, plain, shelf stable",Foundation,2021-10-28,16222,"[{'number': '631', 'name': 'PUFA 22:5 n-3 (DPA..."
196,1999632,"Spinach, baby",Foundation,2021-10-28,100260,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
197,1999633,"Spinach, mature",Foundation,2021-10-28,11457,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
198,2346409,"Strawberries, raw",Foundation,2022-10-28,9316,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."


In [96]:
def get_nutr_df(df):
    food_ids = df['fdcId'].unique()
    vit_min_list = ['fdcId', 'Calcium, Ca', 'Choline, total', 'Folate, total', 'Iron, Fe', 'Magnesium, Mg', 'Niacin', 'Phosphorus, P', 'Potassium, K', 
                'Riboflavin', 'Sodium, Na', 'Thiamin', 'Vitamin A, RAE', 'Vitamin B-12', 'Vitamin B-6', 'Vitamin C, total ascorbic acid',
                'Vitamin D (D2 + D3), International Units', 'Vitamin E (alpha-tocopherol)',  'Vitamin K (phylloquinone)', 'Vitamin K (Dihydrophylloquinone)', 
                'Vitamin K (Menaquinone-4)', 'Zinc, Zn']
    food_nutr = pd.DataFrame(columns=vit_min_list)
    for i in food_ids:
        one_food = df.query(f"fdcId == {i}")[['name', 'amount', 'fdcId']].pivot_table(index = ['fdcId'],columns = 'name',values = 'amount')
        one_food = pd.DataFrame(one_food.to_records())
        cols_to_keep = []
        for i in one_food.columns:
            if (i in vit_min_list):
                cols_to_keep.append(i)
        one_food = one_food[cols_to_keep]
        #print(one_food)
        food_nutr = pd.concat([food_nutr, one_food], axis=0)
        #display(food_nutr)
    return food_nutr

In [97]:
df = pd.json_normalize(myjson, "foodNutrients", ['fdcId'])
food_nutr = get_nutr_df(df)
food_nutr

,fdcId,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,2262074,264.00,NaN,42.50,4.1100,268.00,3.9100,507.00,745.0,NaN,0.996,0.09460,NaN,NaN,0.0838,NaN,NaN,16.700,0.0,NaN,NaN,3.18000
0,2257045,158.00,NaN,0.00,0.1250,8.25,0.0888,19.10,49.3,0.0833,59.200,0.00500,NaN,0.449,0.0000,NaN,63.6,NaN,NaN,NaN,NaN,0.07660
0,1999631,173.00,NaN,0.00,0.2900,6.78,0.0744,29.80,30.8,0.0334,59.600,0.00000,41.4,0.338,0.0000,NaN,37.1,3.320,NaN,NaN,NaN,0.17200
0,2003590,7.10,NaN,3.09,0.0402,4.86,0.0103,5.99,95.9,NaN,4.870,0.00000,NaN,NaN,0.0139,51.2,NaN,NaN,NaN,NaN,NaN,0.00212
0,1750340,5.98,NaN,0.00,0.0150,4.67,0.0906,9.78,104.0,0.0675,1.010,0.00625,NaN,NaN,0.0354,NaN,NaN,NaN,NaN,NaN,NaN,0.01680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1999630,101.00,NaN,19.70,0.5420,21.50,0.2360,68.60,158.0,0.0840,34.300,0.06320,58.2,0.393,0.0552,NaN,27.2,0.156,NaN,NaN,NaN,0.31200
0,1999632,68.40,NaN,116.00,1.2600,92.90,0.5510,39.00,582.0,0.1940,111.000,0.07690,283.0,NaN,0.1950,26.5,NaN,NaN,NaN,NaN,NaN,0.44700
0,1999633,66.60,NaN,113.00,1.0500,93.00,0.5100,40.60,460.0,0.1920,107.000,0.07650,306.0,NaN,0.2140,30.3,NaN,NaN,NaN,NaN,NaN,0.42300
0,2346409,16.90,NaN,NaN,0.2550,12.50,NaN,23.00,161.0,NaN,0.000,NaN,NaN,NaN,NaN,59.6,NaN,NaN,NaN,NaN,NaN,0.11400


In [100]:
food_nutr.query("`Vitamin K (phylloquinone)` > 0")

,fdcId,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,1105073,NaN,NaN,25.0,NaN,NaN,0.570,NaN,NaN,0.000,NaN,0.040,1.0,NaN,0.234,9.7,NaN,NaN,0.20,0.0,0.0,NaN
0,1105314,5.0,NaN,14.0,0.00,28.0,0.662,22.0,326.0,0.000,0.0,0.056,1.0,NaN,0.209,12.3,NaN,NaN,0.10,0.0,0.0,0.16
0,747447,46.0,NaN,65.0,0.69,21.0,0.639,67.0,303.0,0.114,36.0,0.077,8.0,NaN,0.191,91.3,NaN,0.15,102.00,0.0,NaN,0.42
0,746764,33.0,NaN,7.0,0.43,10.8,0.465,31.0,210.0,0.047,66.0,0.039,NaN,NaN,0.088,2.2,NaN,0.69,15.00,0.0,NaN,0.28
0,328637,707.0,NaN,21.0,0.16,26.8,0.052,458.0,77.0,0.441,654.0,0.029,316.0,1.06,0.069,NaN,NaN,0.75,2.40,0.0,9.3,3.67
0,746765,661.0,18.4,NaN,0.18,26.7,0.083,475.0,116.0,0.230,1810.0,0.038,226.0,1.70,0.091,NaN,73.3,0.49,1.60,NaN,5.9,3.28
0,329370,693.0,13.5,26.0,0.20,27.2,0.101,533.0,116.0,0.360,699.0,0.023,203.0,1.65,0.090,NaN,NaN,0.51,1.30,0.0,5.4,3.62
0,325036,884.0,14.1,6.0,0.45,34.9,0.078,634.0,184.0,0.345,1750.0,0.027,228.0,1.35,0.081,NaN,NaN,0.51,1.70,0.0,7.1,4.33
0,325198,866.0,19.2,NaN,0.48,25.1,0.095,540.0,149.0,0.247,1660.0,0.015,300.0,1.45,0.056,NaN,301.0,0.79,3.20,NaN,10.3,2.42
0,746766,224.0,NaN,4.0,0.10,19.7,0.168,162.0,230.0,0.327,105.0,0.043,127.0,0.78,0.099,NaN,NaN,0.27,0.60,0.0,3.1,0.56


## Getting data for Foundation Foods

In [115]:
datatype = 'Foundation'
pagesize=200
pagenum=1
root = 'https://api.nal.usda.gov'

usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={pagenum}&api_key={usda_token}'.format(api_key=usda_token) #establish first endpoint

r = requests.get(root + usda_endpoint,
                    headers = headers)

myjson = json.loads(r.text) # get json of request

food_names_found = pd.json_normalize(myjson)[['fdcId', 'description']] # retrieves fdcId and food name
food_nutr_found = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId'])) # retieves fdcId and nutrition information

#this is going to get the following pages of results
for n in range(2, 3):
    
    usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={n}&api_key={usda_token}'.format(api_key=usda_token)
    r = requests.get(root + usda_endpoint,
                    headers = headers)
    myjson = json.loads(r.text) # get json of request
    next_food_names_found = pd.json_normalize(myjson)[['fdcId', 'description']]
    next_food_nutr_found = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId']))
    food_names_found = pd.concat([food_names_found, next_food_names_found], axis = 0)
    food_nutr_found = pd.concat([food_nutr_found, next_food_nutr_found], axis = 0)


In [116]:
food_names_found

,fdcId,description
0,2262074,"Almond butter, creamy"
1,2257045,"Almond milk, unsweetened, plain, refrigerated"
2,1999631,"Almond milk, unsweetened, plain, shelf stable"
3,2003590,"Apple juice, with added vitamin C, from concen..."
4,1750340,"Apples, fuji, with skin, raw"
...,...,...
5,746785,"Turkey, ground, 93% lean, 7% fat, pan-broiled ..."
6,330137,"Yogurt, Greek, plain, nonfat"
7,2259794,"Yogurt, Greek, plain, whole milk"
8,330415,"Yogurt, Greek, strawberry, nonfat"


In [117]:
food_nutr_found

,fdcId,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,2262074,264.00,NaN,42.50,4.1100,268.00,3.9100,507.00,745.0,NaN,0.996,0.09460,NaN,NaN,0.0838,NaN,NaN,16.70,0.0,NaN,NaN,3.18000
0,2257045,158.00,NaN,0.00,0.1250,8.25,0.0888,19.10,49.3,0.0833,59.200,0.00500,NaN,0.449,0.0000,NaN,63.6,NaN,NaN,NaN,NaN,0.07660
0,1999631,173.00,NaN,0.00,0.2900,6.78,0.0744,29.80,30.8,0.0334,59.600,0.00000,41.4,0.338,0.0000,NaN,37.1,3.32,NaN,NaN,NaN,0.17200
0,2003590,7.10,NaN,3.09,0.0402,4.86,0.0103,5.99,95.9,NaN,4.870,0.00000,NaN,NaN,0.0139,51.2,NaN,NaN,NaN,NaN,NaN,0.00212
0,1750340,5.98,NaN,0.00,0.0150,4.67,0.0906,9.78,104.0,0.0675,1.010,0.00625,NaN,NaN,0.0354,NaN,NaN,NaN,NaN,NaN,NaN,0.01680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,746785,31.00,78.7,NaN,1.5600,29.10,8.0900,259.00,304.0,0.2620,90.000,0.08800,30.0,1.900,0.4970,NaN,NaN,0.18,NaN,NaN,NaN,3.77000
0,330137,111.00,15.1,NaN,0.0700,10.70,0.2110,136.00,141.0,0.2810,36.000,0.02200,1.0,0.700,0.0640,NaN,0.0,0.01,0.0,0.0,0.0,0.53000
0,2259794,111.00,NaN,NaN,0.0000,10.70,0.2270,126.00,147.0,0.2440,33.800,0.05520,NaN,NaN,0.0439,NaN,0.0,NaN,NaN,NaN,NaN,0.47300
0,330415,97.00,NaN,20.00,0.0900,10.40,0.2080,113.00,133.0,0.2500,32.000,NaN,0.0,0.450,0.0500,0.2,40.0,NaN,NaN,NaN,NaN,0.46000


## Getting data for Survey Foods

In [118]:
datatype = 'Survey (FNDDS)'
pagesize=200
pagenum=1
root = 'https://api.nal.usda.gov'

usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={pagenum}&api_key={usda_token}'.format(api_key=usda_token) #establish first endpoint

r = requests.get(root + usda_endpoint,
                    headers = headers)

myjson = json.loads(r.text) # get json of request

food_names_surv = pd.json_normalize(myjson)[['fdcId', 'description']] # retrieves fdcId and food name
food_nutr_surv = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId'])) # retieves fdcId and nutrition information

#this is going to get the following pages of results
for n in range(2, 30):
    
    usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={n}&api_key={usda_token}'.format(api_key=usda_token)
    r = requests.get(root + usda_endpoint,
                    headers = headers)
    myjson = json.loads(r.text) # get json of request
    next_food_names_surv = pd.json_normalize(myjson)[['fdcId', 'description']]
    next_food_nutr_surv = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId']))
    food_names_surv = pd.concat([food_names_surv, next_food_names_surv], axis = 0)
    food_nutr_surv = pd.concat([food_nutr_surv, next_food_nutr_surv], axis = 0)

In [119]:
food_names_surv

,fdcId,description
0,2341752,Abalone
1,2344293,"Adobo, with noodles"
2,2344443,"Adobo, with rice"
3,2345842,Agave liquid sweetener
4,2346197,Alcoholic malt beverage
...,...,...
19,2340799,"Yogurt, whole milk, plain"
20,2345875,Yokan
21,2345065,Yuca fries
22,2345631,"Zucchini, pickled"


In [138]:
food_nutr_surv

,fdcId,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,2341752,39.0,80.7,5.0,3.97,60.0,1.770,236.0,310.0,0.118,574.0,0.201,2.0,0.77,0.168,2.0,NaN,5.80,34.2,NaN,NaN,1.02
0,2344293,15.0,57.1,3.0,0.99,18.0,3.060,149.0,200.0,0.204,510.0,0.192,4.0,0.43,0.216,0.0,NaN,0.20,0.0,NaN,NaN,2.39
0,2344443,18.0,59.2,21.0,1.34,21.0,3.590,161.0,216.0,0.213,522.0,0.242,4.0,0.44,0.250,0.0,NaN,0.21,0.0,NaN,NaN,2.54
0,2345842,1.0,13.3,30.0,0.09,1.0,0.689,1.0,4.0,0.165,4.0,0.122,8.0,0.00,0.234,17.0,NaN,0.98,22.5,NaN,NaN,0.01
0,2346197,8.0,10.1,18.0,0.03,12.0,1.100,32.0,62.0,0.090,4.0,0.030,0.0,0.02,0.090,0.0,NaN,0.00,0.0,NaN,NaN,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2340799,121.0,15.2,7.0,0.05,12.0,0.075,95.0,155.0,0.142,46.0,0.029,27.0,0.37,0.032,0.5,NaN,0.06,0.2,NaN,NaN,0.59
0,2345875,15.0,14.1,10.0,0.59,14.0,0.215,49.0,120.0,0.035,119.0,0.046,0.0,0.00,0.035,0.3,NaN,0.01,1.9,NaN,NaN,0.29
0,2345065,15.0,21.6,22.0,0.27,19.0,0.740,25.0,247.0,0.042,154.0,0.063,1.0,0.00,0.076,15.0,NaN,2.39,16.7,NaN,NaN,0.31
0,2345631,15.0,7.9,17.0,0.31,15.0,0.355,32.0,217.0,0.075,951.0,0.034,8.0,0.00,0.122,12.6,NaN,0.10,3.6,NaN,NaN,0.27


## Getting data for legacy foods

In [121]:
datatype = 'SR Legacy'
pagesize=200
pagenum=1
root = 'https://api.nal.usda.gov'

usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={pagenum}&api_key={usda_token}'.format(api_key=usda_token) #establish first endpoint

r = requests.get(root + usda_endpoint,
                    headers = headers)

myjson = json.loads(r.text) # get json of request

food_names_leg = pd.json_normalize(myjson)[['fdcId', 'description']] # retrieves fdcId and food name
food_nutr_leg = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId'])) # retieves fdcId and nutrition information

#this is going to get the following pages of results
for n in range(2, 40):
    
    usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={n}&api_key={usda_token}'.format(api_key=usda_token)
    r = requests.get(root + usda_endpoint,
                    headers = headers)
    myjson = json.loads(r.text) # get json of request
    next_food_names_leg = pd.json_normalize(myjson)[['fdcId', 'description']]
    next_food_nutr_leg = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId']))
    food_names_leg = pd.concat([food_names_leg, next_food_names_leg], axis = 0)
    food_nutr_leg = pd.concat([food_nutr_leg, next_food_nutr_leg], axis = 0)

In [122]:
food_names_leg

,fdcId,description
0,167782,"Abiyuch, raw"
1,171687,"Acerola juice, raw"
2,171686,"Acerola, (west indian cherry), raw"
3,168061,Acorn stew (Apache)
4,168992,"Agave, cooked (Southwest)"
...,...,...
188,172219,"Yogurt, vanilla, low fat, fortified with vitam..."
189,170888,"Yogurt, vanilla, low fat."
190,170906,"Yogurt, vanilla, non-fat"
191,173730,"Yokan, prepared from adzuki beans and sugar"


In [123]:
food_nutr_leg

,fdcId,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,167782,8.0,NaN,NaN,1.61,24.0,NaN,47.0,304.0,NaN,20.0,NaN,5.0,NaN,NaN,54.1,NaN,NaN,NaN,NaN,NaN,0.31
0,171687,10.0,NaN,14.0,0.50,12.0,0.400,9.0,97.0,0.060,3.0,0.020,25.0,0.00,0.004,1600.0,NaN,0.18,1.4,NaN,NaN,0.10
0,171686,12.0,NaN,14.0,0.20,18.0,0.400,11.0,146.0,0.060,7.0,0.020,38.0,0.00,0.009,1680.0,NaN,NaN,NaN,NaN,NaN,0.10
0,168061,14.0,NaN,33.0,1.00,12.0,2.140,62.0,110.0,0.125,130.0,0.175,0.0,0.68,0.055,0.0,NaN,0.30,0.0,0.0,0.9,1.60
0,168992,460.0,8.8,3.0,3.55,39.0,0.162,9.0,59.0,0.099,13.0,0.012,6.0,0.00,0.087,0.3,0.0,0.36,4.9,0.0,NaN,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,172219,171.0,15.2,11.0,0.07,16.0,0.107,135.0,219.0,0.201,66.0,0.042,12.0,0.53,0.045,0.8,47.0,0.02,0.1,NaN,NaN,0.83
0,170888,171.0,15.2,11.0,0.07,16.0,0.107,135.0,219.0,0.201,66.0,0.042,12.0,0.53,0.045,0.8,1.0,0.02,0.1,NaN,NaN,0.83
0,170906,118.0,15.6,11.0,0.00,16.0,0.107,88.0,141.0,0.200,47.0,0.042,61.0,0.53,0.045,0.0,35.0,0.01,0.0,NaN,NaN,0.83
0,173730,27.0,NaN,8.0,1.16,18.0,0.057,40.0,45.0,0.004,83.0,0.005,0.0,0.00,0.008,0.0,0.0,NaN,NaN,NaN,NaN,0.07


## getting data for branded foods

In [144]:
datatype = 'Branded'
pagesize=200
pagenum=1
root = 'https://api.nal.usda.gov'

usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={pagenum}&api_key={usda_token}'.format(api_key=usda_token) #establish first endpoint

r = requests.get(root + usda_endpoint,
                    headers = headers)

myjson = json.loads(r.text) # get json of request

food_names_brand = pd.json_normalize(myjson)[['fdcId', 'description', 'brandOwner']] # retrieves fdcId and food name
food_nutr_brand = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId'])) # retieves fdcId and nutrition information

#this is going to get the following pages of results
for n in range(2, 51):
    #print(n)
    usda_endpoint = f'/fdc/v1/foods/list?dataType={datatype}&pageSize={pagesize}&pageNumber={n}&api_key={usda_token}'.format(api_key=usda_token)
    r = requests.get(root + usda_endpoint,
                    headers = headers)
    myjson = json.loads(r.text) # get json of request
    next_food_names_brand = pd.json_normalize(myjson)[['fdcId', 'description', 'brandOwner']]
    next_food_nutr_brand = get_nutr_df(pd.json_normalize(myjson, "foodNutrients", ['fdcId']))
    food_names_brand = pd.concat([food_names_brand, next_food_names_brand], axis = 0)
    food_nutr_brand = pd.concat([food_nutr_brand, next_food_nutr_brand], axis = 0)

In [145]:
food_names_brand

,fdcId,description,brandOwner
0,1944909,"ARTISAN SMOOTH & CREAMY GELATO, TOASTED COCONUT",Lunds Inc
1,1856309,"BARBECUE SAUCE, HONEY","Lowe's Food Stores, Inc."
2,1861692,BERRY NUT BLEND BREAKFAST IN THE GO!,"Snyder's-Lance, Inc."
3,2216356,"CHOCOLATE CAKE HOOPIE PIES, CHOCOLATE CAKE","US Foods, LLC"
4,1909487,"COCONUT WATER BARS, MANGO",C2O LLC
...,...,...,...
195,1879846,"AHOLD, SNACKING CHOCOLATE","Ahold USA, Inc."
196,1862535,"AHOLD, SOUTHWEST STYLE SALSA, MEDIUM","Ahold USA, Inc."
197,1858673,"AHOLD, SPINACH DIP","Ahold USA, Inc."
198,1858594,"AHOLD, SPLIT TOP WHITE ENRICHED BREAD","Ahold USA, Inc."


In [146]:
food_nutr_brand

,fdcId,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,1944909,123.0,NaN,NaN,0.17,NaN,NaN,NaN,170.0,NaN,52.0,NaN,NaN,NaN,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN
0,1856309,0.0,NaN,NaN,0.00,NaN,NaN,NaN,27.0,NaN,919.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,1861692,188.0,NaN,NaN,2.54,NaN,NaN,NaN,NaN,NaN,153.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,2216356,27.0,NaN,NaN,4.78,NaN,NaN,NaN,NaN,NaN,520.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,1909487,0.0,NaN,NaN,0.00,18.0,0.899,449.0,213.0,NaN,62.0,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1879846,140.0,NaN,NaN,4.19,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,1862535,62.0,NaN,NaN,1.12,NaN,NaN,NaN,NaN,NaN,531.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,1858673,67.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,417.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,1858594,71.0,NaN,114.0,2.57,NaN,4.290,NaN,NaN,0.364,500.0,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


## Making the big table!

In [149]:
nutr_found = pd.merge(food_names_found, food_nutr_found,
                        on = ['fdcId'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')
nutr_found['matched'].value_counts()

both          210
left_only       0
right_only      0
Name: matched, dtype: int64

In [150]:
nutr_surv = pd.merge(food_names_surv, food_nutr_surv,
                        on = ['fdcId'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')
nutr_surv['matched'].value_counts()

both          5624
left_only        0
right_only       0
Name: matched, dtype: int64

In [151]:
nutr_leg = pd.merge(food_names_leg, food_nutr_leg,
                        on = ['fdcId'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')
nutr_leg['matched'].value_counts()

both          7793
left_only        0
right_only       0
Name: matched, dtype: int64

In [152]:
nutr_brand = pd.merge(food_names_brand[['fdcId','description']], food_nutr_brand,
                        on = ['fdcId'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')
nutr_brand['matched'].value_counts()

both          9750
left_only      250
right_only       0
Name: matched, dtype: int64

In [158]:
food_brand = pd.merge(nutr_brand.query("matched == 'left_only'")[['fdcId']], food_names_brand,
                      on = ['fdcId'],
                      how = 'outer',
                      validate = 'one_to_one',
                      indicator = 'matched')
food_brand = food_brand.query("matched == 'right_only'")[['fdcId', 'brandOwner']]
food_brand

/tmp/ipykernel_43/3676110890.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  food_brand = pd.merge(nutr_brand.query("matched == 'left_only'")[['fdcId']], food_names_brand,


,fdcId,brandOwner
250,1944909.0,Lunds Inc
251,1856309.0,"Lowe's Food Stores, Inc."
252,1861692.0,"Snyder's-Lance, Inc."
253,2216356.0,"US Foods, LLC"
254,1909487.0,C2O LLC
...,...,...
9995,1879846.0,"Ahold USA, Inc."
9996,1862535.0,"Ahold USA, Inc."
9997,1858673.0,"Ahold USA, Inc."
9998,1858594.0,"Ahold USA, Inc."


In [159]:
food_brand = food_brand.astype({"fdcId": int})
food_brand

,fdcId,brandOwner
250,1944909,Lunds Inc
251,1856309,"Lowe's Food Stores, Inc."
252,1861692,"Snyder's-Lance, Inc."
253,2216356,"US Foods, LLC"
254,1909487,C2O LLC
...,...,...
9995,1879846,"Ahold USA, Inc."
9996,1862535,"Ahold USA, Inc."
9997,1858673,"Ahold USA, Inc."
9998,1858594,"Ahold USA, Inc."


In [160]:
nutr_brand = nutr_brand.query("matched == 'both'").drop(['matched'], axis = 1)
nutr_brand

,fdcId,description,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,1944909,"ARTISAN SMOOTH & CREAMY GELATO, TOASTED COCONUT",123.0,NaN,NaN,0.17,NaN,NaN,NaN,170.0,NaN,52.0,NaN,NaN,NaN,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN
1,1856309,"BARBECUE SAUCE, HONEY",0.0,NaN,NaN,0.00,NaN,NaN,NaN,27.0,NaN,919.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1861692,BERRY NUT BLEND BREAKFAST IN THE GO!,188.0,NaN,NaN,2.54,NaN,NaN,NaN,NaN,NaN,153.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2216356,"CHOCOLATE CAKE HOOPIE PIES, CHOCOLATE CAKE",27.0,NaN,NaN,4.78,NaN,NaN,NaN,NaN,NaN,520.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1909487,"COCONUT WATER BARS, MANGO",0.0,NaN,NaN,0.00,18.0,0.899,449.0,213.0,NaN,62.0,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1879846,"AHOLD, SNACKING CHOCOLATE",140.0,NaN,NaN,4.19,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9996,1862535,"AHOLD, SOUTHWEST STYLE SALSA, MEDIUM",62.0,NaN,NaN,1.12,NaN,NaN,NaN,NaN,NaN,531.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9997,1858673,"AHOLD, SPINACH DIP",67.0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,417.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9998,1858594,"AHOLD, SPLIT TOP WHITE ENRICHED BREAD",71.0,NaN,114.0,2.57,NaN,4.290,NaN,NaN,0.364,500.0,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [265]:
food = pd.concat([nutr_found, nutr_surv, nutr_leg], axis = 0).drop(['matched'], axis = 1) #, nutr_brand
food

,fdcId,description,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,2262074,"Almond butter, creamy",264.00,NaN,42.50,4.1100,268.00,3.9100,507.00,745.0,NaN,0.996,0.09460,NaN,NaN,0.0838,NaN,NaN,16.70,0.0,NaN,NaN,3.18000
1,2257045,"Almond milk, unsweetened, plain, refrigerated",158.00,NaN,0.00,0.1250,8.25,0.0888,19.10,49.3,0.0833,59.200,0.00500,NaN,0.449,0.0000,NaN,63.6,NaN,NaN,NaN,NaN,0.07660
2,1999631,"Almond milk, unsweetened, plain, shelf stable",173.00,NaN,0.00,0.2900,6.78,0.0744,29.80,30.8,0.0334,59.600,0.00000,41.4,0.338,0.0000,NaN,37.1,3.32,NaN,NaN,NaN,0.17200
3,2003590,"Apple juice, with added vitamin C, from concen...",7.10,NaN,3.09,0.0402,4.86,0.0103,5.99,95.9,NaN,4.870,0.00000,NaN,NaN,0.0139,51.2,NaN,NaN,NaN,NaN,NaN,0.00212
4,1750340,"Apples, fuji, with skin, raw",5.98,NaN,0.00,0.0150,4.67,0.0906,9.78,104.0,0.0675,1.010,0.00625,NaN,NaN,0.0354,NaN,NaN,NaN,NaN,NaN,NaN,0.01680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,172219,"Yogurt, vanilla, low fat, fortified with vitam...",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,47.0,0.02,0.1,NaN,NaN,0.83000
7789,170888,"Yogurt, vanilla, low fat.",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,1.0,0.02,0.1,NaN,NaN,0.83000
7790,170906,"Yogurt, vanilla, non-fat",118.00,15.6,11.00,0.0000,16.00,0.1070,88.00,141.0,0.2000,47.000,0.04200,61.0,0.530,0.0450,0.0,35.0,0.01,0.0,NaN,NaN,0.83000
7791,173730,"Yokan, prepared from adzuki beans and sugar",27.00,NaN,8.00,1.1600,18.00,0.0570,40.00,45.0,0.0040,83.000,0.00500,0.0,0.000,0.0080,0.0,0.0,NaN,NaN,NaN,NaN,0.07000


In [266]:
food = food.fillna(0)
food

,fdcId,description,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Vitamin K (Dihydrophylloquinone),Vitamin K (Menaquinone-4),"Zinc, Zn"
0,2262074,"Almond butter, creamy",264.00,0.0,42.50,4.1100,268.00,3.9100,507.00,745.0,0.0000,0.996,0.09460,0.0,0.000,0.0838,0.0,0.0,16.70,0.0,0.0,0.0,3.18000
1,2257045,"Almond milk, unsweetened, plain, refrigerated",158.00,0.0,0.00,0.1250,8.25,0.0888,19.10,49.3,0.0833,59.200,0.00500,0.0,0.449,0.0000,0.0,63.6,0.00,0.0,0.0,0.0,0.07660
2,1999631,"Almond milk, unsweetened, plain, shelf stable",173.00,0.0,0.00,0.2900,6.78,0.0744,29.80,30.8,0.0334,59.600,0.00000,41.4,0.338,0.0000,0.0,37.1,3.32,0.0,0.0,0.0,0.17200
3,2003590,"Apple juice, with added vitamin C, from concen...",7.10,0.0,3.09,0.0402,4.86,0.0103,5.99,95.9,0.0000,4.870,0.00000,0.0,0.000,0.0139,51.2,0.0,0.00,0.0,0.0,0.0,0.00212
4,1750340,"Apples, fuji, with skin, raw",5.98,0.0,0.00,0.0150,4.67,0.0906,9.78,104.0,0.0675,1.010,0.00625,0.0,0.000,0.0354,0.0,0.0,0.00,0.0,0.0,0.0,0.01680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,172219,"Yogurt, vanilla, low fat, fortified with vitam...",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,47.0,0.02,0.1,0.0,0.0,0.83000
7789,170888,"Yogurt, vanilla, low fat.",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,1.0,0.02,0.1,0.0,0.0,0.83000
7790,170906,"Yogurt, vanilla, non-fat",118.00,15.6,11.00,0.0000,16.00,0.1070,88.00,141.0,0.2000,47.000,0.04200,61.0,0.530,0.0450,0.0,35.0,0.01,0.0,0.0,0.0,0.83000
7791,173730,"Yokan, prepared from adzuki beans and sugar",27.00,0.0,8.00,1.1600,18.00,0.0570,40.00,45.0,0.0040,83.000,0.00500,0.0,0.000,0.0080,0.0,0.0,0.00,0.0,0.0,0.0,0.07000


In [267]:
food['Vitamin K'] = food['Vitamin K (phylloquinone)'] + food['Vitamin K (Dihydrophylloquinone)'] + food['Vitamin K (Menaquinone-4)']
food = food.drop(['Vitamin K (phylloquinone)', 'Vitamin K (Dihydrophylloquinone)', 'Vitamin K (Menaquinone-4)'], axis = 1)
food

,fdcId,description,"Calcium, Ca","Choline, total","Folate, total","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,"Sodium, Na",Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid","Vitamin D (D2 + D3), International Units",Vitamin E (alpha-tocopherol),"Zinc, Zn",Vitamin K
0,2262074,"Almond butter, creamy",264.00,0.0,42.50,4.1100,268.00,3.9100,507.00,745.0,0.0000,0.996,0.09460,0.0,0.000,0.0838,0.0,0.0,16.70,3.18000,0.0
1,2257045,"Almond milk, unsweetened, plain, refrigerated",158.00,0.0,0.00,0.1250,8.25,0.0888,19.10,49.3,0.0833,59.200,0.00500,0.0,0.449,0.0000,0.0,63.6,0.00,0.07660,0.0
2,1999631,"Almond milk, unsweetened, plain, shelf stable",173.00,0.0,0.00,0.2900,6.78,0.0744,29.80,30.8,0.0334,59.600,0.00000,41.4,0.338,0.0000,0.0,37.1,3.32,0.17200,0.0
3,2003590,"Apple juice, with added vitamin C, from concen...",7.10,0.0,3.09,0.0402,4.86,0.0103,5.99,95.9,0.0000,4.870,0.00000,0.0,0.000,0.0139,51.2,0.0,0.00,0.00212,0.0
4,1750340,"Apples, fuji, with skin, raw",5.98,0.0,0.00,0.0150,4.67,0.0906,9.78,104.0,0.0675,1.010,0.00625,0.0,0.000,0.0354,0.0,0.0,0.00,0.01680,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,172219,"Yogurt, vanilla, low fat, fortified with vitam...",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,47.0,0.02,0.83000,0.1
7789,170888,"Yogurt, vanilla, low fat.",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,1.0,0.02,0.83000,0.1
7790,170906,"Yogurt, vanilla, non-fat",118.00,15.6,11.00,0.0000,16.00,0.1070,88.00,141.0,0.2000,47.000,0.04200,61.0,0.530,0.0450,0.0,35.0,0.01,0.83000,0.0
7791,173730,"Yokan, prepared from adzuki beans and sugar",27.00,0.0,8.00,1.1600,18.00,0.0570,40.00,45.0,0.0040,83.000,0.00500,0.0,0.000,0.0080,0.0,0.0,0.00,0.07000,0.0


In [268]:
food = food.rename({'Calcium, Ca':'calcium',
                    'Choline, total':'choline',
                    'Folate, total':'folate',
                    'Iron, Fe':'iron',
                    'Magnesium, Mg':'magnesium',
                    'Niacin':'niacin',
                    'Phosphorus, P':'phosphorus',
                    'Potassium, K':'potassium',
                    'Riboflavin':'riboflavin',
                    'Sodium, Na':'sodium',
                    'Thiamin':'thiamin',
                    'Vitamin A, RAE':'vit_a',
                    'Vitamin B-12':'vit_b12',
                    'Vitamin B-6':'vit_b6',
                    'Vitamin C, total ascorbic acid':'vit_c',
                    'Vitamin D (D2 + D3), International Units':'vit_d',
                    'Vitamin E (alpha-tocopherol)':'vit_e',
                    'Zinc, Zn':'zinc',
                    'Vitamin K':'vit_k'},
                            axis=1)
food

,fdcId,description,calcium,choline,folate,iron,magnesium,niacin,phosphorus,potassium,riboflavin,sodium,thiamin,vit_a,vit_b12,vit_b6,vit_c,vit_d,vit_e,zinc,vit_k
0,2262074,"Almond butter, creamy",264.00,0.0,42.50,4.1100,268.00,3.9100,507.00,745.0,0.0000,0.996,0.09460,0.0,0.000,0.0838,0.0,0.0,16.70,3.18000,0.0
1,2257045,"Almond milk, unsweetened, plain, refrigerated",158.00,0.0,0.00,0.1250,8.25,0.0888,19.10,49.3,0.0833,59.200,0.00500,0.0,0.449,0.0000,0.0,63.6,0.00,0.07660,0.0
2,1999631,"Almond milk, unsweetened, plain, shelf stable",173.00,0.0,0.00,0.2900,6.78,0.0744,29.80,30.8,0.0334,59.600,0.00000,41.4,0.338,0.0000,0.0,37.1,3.32,0.17200,0.0
3,2003590,"Apple juice, with added vitamin C, from concen...",7.10,0.0,3.09,0.0402,4.86,0.0103,5.99,95.9,0.0000,4.870,0.00000,0.0,0.000,0.0139,51.2,0.0,0.00,0.00212,0.0
4,1750340,"Apples, fuji, with skin, raw",5.98,0.0,0.00,0.0150,4.67,0.0906,9.78,104.0,0.0675,1.010,0.00625,0.0,0.000,0.0354,0.0,0.0,0.00,0.01680,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,172219,"Yogurt, vanilla, low fat, fortified with vitam...",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,47.0,0.02,0.83000,0.1
7789,170888,"Yogurt, vanilla, low fat.",171.00,15.2,11.00,0.0700,16.00,0.1070,135.00,219.0,0.2010,66.000,0.04200,12.0,0.530,0.0450,0.8,1.0,0.02,0.83000,0.1
7790,170906,"Yogurt, vanilla, non-fat",118.00,15.6,11.00,0.0000,16.00,0.1070,88.00,141.0,0.2000,47.000,0.04200,61.0,0.530,0.0450,0.0,35.0,0.01,0.83000,0.0
7791,173730,"Yokan, prepared from adzuki beans and sugar",27.00,0.0,8.00,1.1600,18.00,0.0570,40.00,45.0,0.0040,83.000,0.00500,0.0,0.000,0.0080,0.0,0.0,0.00,0.07000,0.0


# Final Tables

In [235]:
demo

,SEQN,sex,age_year,race,education,marital_status,income_pov_rat
0,109263.0,M,2.0,Non-Hispanic Asian,NaN,NaN,4.66
1,109264.0,F,13.0,Mexican American,NaN,NaN,0.83
2,109265.0,M,2.0,Non-Hispanic White,NaN,NaN,3.06
3,109266.0,F,29.0,Non-Hispanic Asian,College graduate or above,Never married,5.00
4,109269.0,M,2.0,Other Hispanic,NaN,NaN,0.96
...,...,...,...,...,...,...,...
12394,112157.0,M,3.0,Mexican American,NaN,NaN,0.48
12399,115293.0,F,2.0,Non-Hispanic White,NaN,NaN,0.99
12403,117889.0,F,2.0,Other Hispanic,NaN,NaN,0.85
12407,122957.0,F,2.0,Non-Hispanic Black,NaN,NaN,1.47


In [259]:
demo.to_csv('demo.csv', index=False)

In [236]:
pop_nutr

,SEQN,avg_vit_e,avg_vit_a,avg_thiamin_b1,avg_riboflavin_b2,avg_niacin,avg_vit_b6,avg_folate,avg_choline,avg_vit_b12,avg_vit_c,avg_vit_d,avg_vit_k,avg_calcium,avg_phosphorus,avg_magnesium,avg_iron,avg_zinc,avg_sodium,avg_potassium
0,109263.0,4.645,304.5,1.6960,1.0150,15.4385,1.1695,422.0,139.15,2.345,167.45,230.0,35.30,1026.5,758.5,160.0,10.615,6.395,1923.0,1561.5
1,109264.0,5.100,1122.5,2.2000,1.8770,36.1480,3.5925,659.0,198.15,8.880,44.90,146.0,118.75,571.5,1055.5,196.5,28.245,16.430,2195.0,1381.0
2,109265.0,4.440,633.0,1.1845,1.5540,16.4525,1.4330,239.0,197.65,4.310,66.25,244.0,57.50,1112.0,1192.0,196.0,7.230,7.320,1991.0,2279.0
3,109266.0,11.050,420.0,1.2930,1.6440,12.5290,1.3310,282.0,223.15,2.430,79.50,162.0,91.20,1048.0,1246.5,342.0,11.550,7.635,2720.0,2404.0
4,109269.0,4.075,280.0,0.5835,0.8115,5.4995,0.3315,157.0,268.25,1.260,128.05,106.0,55.25,358.5,548.0,75.5,5.860,3.040,1503.5,1081.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,112157.0,6.410,555.0,2.7330,2.7110,43.5520,4.3110,560.0,215.50,9.870,43.80,328.0,38.20,785.0,1006.0,250.0,29.710,8.140,2726.0,2179.0
12399,115293.0,4.320,410.0,1.1100,2.1990,13.5170,0.8230,236.0,222.30,5.420,14.60,600.0,51.30,1459.0,1343.0,198.0,5.940,6.550,1677.0,2458.0
12403,117889.0,4.460,463.0,1.0850,1.4080,13.1980,1.1280,257.0,232.00,2.760,64.80,152.0,30.20,897.0,938.0,170.0,10.680,7.550,1721.0,1469.0
12407,122957.0,4.170,266.0,0.9370,1.1750,13.4810,1.1240,163.0,156.80,2.310,29.40,140.0,25.60,792.0,1030.0,201.0,7.260,6.240,2009.0,1718.0


In [260]:
pop_nutr.to_csv('pop_nutr.csv', index=False)

In [237]:
rec

,age_lower,age_upper,sex,vit_a,vit_e,vit_d,vit_c,thiamin_b1,riboflavin_b2,niacin,vit_b6,vit_b12,choline,vit_k,folate,calcium,iron,magnesium,phosphorus,potassium,sodium,zinc
0,2,3,M/F,300,6,600,15,0.5,0.5,6,0.5,0.9,200,30,150,700,7,80,460,2000,1200,3
1,4,8,F,400,7,600,25,0.6,0.6,8,0.6,1.2,250,55,200,1000,10,130,500,2300,1500,5
2,9,13,F,600,11,600,45,0.9,0.9,12,1.0,1.8,375,60,300,1300,8,240,1250,2300,1800,8
3,14,18,F,700,15,600,65,1.0,1.0,14,1.2,2.4,400,75,400,1300,15,360,1250,2300,2300,9
4,19,30,F,700,15,600,75,1.1,1.1,14,1.3,2.4,425,90,400,1000,18,310,700,2600,2300,8
5,31,50,F,700,15,600,75,1.1,1.1,14,1.3,2.4,425,90,400,1000,18,320,700,2600,2300,8
6,51,150,F,700,15,600,75,1.1,1.1,14,1.5,2.4,425,90,400,1200,8,320,700,2600,2300,8
7,4,8,M,400,7,600,25,0.6,0.6,8,0.6,1.2,250,55,200,1000,10,130,500,2300,1500,5
8,9,13,M,600,11,600,45,0.9,0.9,12,1.0,1.8,375,60,300,1300,8,240,1250,2500,1800,8
9,14,18,M,900,15,600,75,1.2,1.3,16,1.3,2.4,550,75,400,1300,11,410,1250,3000,2300,11


In [261]:
rec.to_csv('rec.csv', index=False)

In [256]:
food

,fdcId,description,calcium,choline,folate,iron,magnesium,niacin,phosphorus,potassium,riboflavin,sodium,thiamin,vit_a,vit_b12,vit_b6,vit_c,vit_d,vit_e,zinc,vit_k
0,2262074,"Almond butter, creamy",264.00,0.0,42.50,4.1100,268.00,3.9100,507.00,745.0,0.0000,0.996,0.09460,0.0,0.000,0.0838,0.0,0.0,16.70,3.18000,0.0
1,2257045,"Almond milk, unsweetened, plain, refrigerated",158.00,0.0,0.00,0.1250,8.25,0.0888,19.10,49.3,0.0833,59.200,0.00500,0.0,0.449,0.0000,0.0,63.6,0.00,0.07660,0.0
2,1999631,"Almond milk, unsweetened, plain, shelf stable",173.00,0.0,0.00,0.2900,6.78,0.0744,29.80,30.8,0.0334,59.600,0.00000,41.4,0.338,0.0000,0.0,37.1,3.32,0.17200,0.0
3,2003590,"Apple juice, with added vitamin C, from concen...",7.10,0.0,3.09,0.0402,4.86,0.0103,5.99,95.9,0.0000,4.870,0.00000,0.0,0.000,0.0139,51.2,0.0,0.00,0.00212,0.0
4,1750340,"Apples, fuji, with skin, raw",5.98,0.0,0.00,0.0150,4.67,0.0906,9.78,104.0,0.0675,1.010,0.00625,0.0,0.000,0.0354,0.0,0.0,0.00,0.01680,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1879846,"AHOLD, SNACKING CHOCOLATE",140.00,0.0,0.00,4.1900,0.00,0.0000,0.00,0.0,0.0000,23.000,0.00000,0.0,0.000,0.0000,0.0,0.0,0.00,0.00000,0.0
9996,1862535,"AHOLD, SOUTHWEST STYLE SALSA, MEDIUM",62.00,0.0,0.00,1.1200,0.00,0.0000,0.00,0.0,0.0000,531.000,0.00000,0.0,0.000,0.0000,0.0,0.0,0.00,0.00000,0.0
9997,1858673,"AHOLD, SPINACH DIP",67.00,0.0,0.00,0.0000,0.00,0.0000,0.00,0.0,0.0000,417.000,0.00000,0.0,0.000,0.0000,0.0,0.0,0.00,0.00000,0.0
9998,1858594,"AHOLD, SPLIT TOP WHITE ENRICHED BREAD",71.00,0.0,114.00,2.5700,0.00,4.2900,0.00,0.0,0.3640,500.000,1.00000,0.0,0.000,0.0000,0.0,0.0,0.00,0.00000,0.0


In [269]:
food.to_csv('food.csv', index=False)

In [239]:
food_brand

,fdcId,brandOwner
250,1944909,Lunds Inc
251,1856309,"Lowe's Food Stores, Inc."
252,1861692,"Snyder's-Lance, Inc."
253,2216356,"US Foods, LLC"
254,1909487,C2O LLC
...,...,...
9995,1879846,"Ahold USA, Inc."
9996,1862535,"Ahold USA, Inc."
9997,1858673,"Ahold USA, Inc."
9998,1858594,"Ahold USA, Inc."


In [263]:
food_brand.to_csv('food_brand.csv', index=False)

In [246]:
units

,units
vit_a,mcg RAE
vit_e,mg AT
vit_d,IU
vit_c,mg
thiamin_b1,mg
riboflavin_b2,mg
niacin,mg
vit_b6,mg
vit_b12,mcg
choline,mg


In [264]:
units.to_csv('units.csv', index=False)